In [1]:
import torch
import torch.nn as nn

import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from Define_Model.LossFunction import CenterLoss

In [2]:
torch.cuda.is_available()

True

In [3]:
enr_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/spkvecs.npy')
enr_len = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/veclabs.npy')


In [4]:
end_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/uttvecs.npy')
ned_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/uttvecs.npy')

end_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/answer.npy')
ned_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/answer.npy')+51

In [5]:
# Train set
enr_vec_t = enr_vec.transpose()
print("The shape of enrollment vectors are ", enr_vec_t.shape)

# avg_enr_vec = [] #avg vectors for centers
enr_lab = []
for i in range(1, len(enr_len)):
#     avg_enr_vec.append(enr_cos_t[enr_len[i-1]:enr_len[i],:].mean(axis=0).reshape(1,-1))
    for x in range(enr_len[i]-enr_len[i-1]):
        enr_lab.append(i-1)
print("The shape of enrolled vectors' labels :", enr_lab.__len__())

The shape of enrollment vectors are  (1392, 128)
The shape of enrolled vectors' labels : 1392


In [6]:
# Test sets
end_vec_t = end_vec.transpose()
print("The shape of enrolled vectors are ", end_vec.shape)

ned_vec_t = ned_vec.transpose()
print("The shape of not enrolled vectors are ", ned_vec.shape)


The shape of enrolled vectors are  (31487, 128)
The shape of not enrolled vectors are  (11699, 128)


In [7]:
# avg_center = np.concatenate(avg_enr_vec, axis=0).transpose()
# print(avg_center.shape)


In [11]:
# print(ned_cls.shape)
# enr_cos = np.matmul(enr_cos_t, avg_center)

# train 
# data and label
# x = np.append(enr_vec_t, ned_vec[:100, :], axis=0)
# y = np.append(enr_lab, ned_cls[:100])

# no out set vectors
x = enr_vec_t
y = np.array(enr_lab) #, ned_cls[:100])


In [12]:
# test 
# data and label
# pre_x = np.append(end_vec, ned_vec[100:], axis=0)
# print(pre_x.shape)

# real_y = np.append(end_cls, ned_cls[100:])
# print(real_y.shape)

pre_x = end_vec #, ned_vec[100:], axis=0)
print(pre_x.shape)

real_y = end_cls#, ned_cls[100:])
print(real_y.shape)


(31487, 128)
(31487,)


In [13]:
xlf = DecisionTreeClassifier(random_state=0)
x = x / np.sqrt(np.power(x, 2).sum(axis=1)).reshape(-1,1) * 12
xlf.fit(x, y)

pre_x = pre_x / np.sqrt(np.power(pre_x, 2).sum(axis=1)).reshape(-1,1) * 12
pred_y = xlf.predict(pre_x)

correct = 0
for m,n in zip(pred_y, real_y):
    if m==n:
        correct +=1
acc = correct / len(real_y)
print(acc *100, "%") 
#cos: 20.192173791950982%
#vec: 39.0256 

39.025629624924576 %


In [16]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F

# linearpred = nn.Sequential(nn.Linear(128, 64), # 使用cos分数预测
#                            nn.BatchNorm1d(64),
#                            nn.ReLU6(),
#                            nn.Linear(64, 51))
linearpred = nn.Linear(128, 50)

optimizer = optim.SGD(linearpred.parameters() ,lr=0.1,
                momentum=0.9,
                dampening=0,
                weight_decay=0.001)

scheduler = MultiStepLR(optimizer, milestones=[20, 20, 20], gamma=0.1)
output_softmax = nn.Softmax(dim=1)
crossentropy = nn.CrossEntropyLoss()
center_loss = CenterLoss(num_classes=50, feat_dim=128).cuda()

In [17]:
epoch = 80
t_x = torch.tensor(x).cuda()
t_x = F.normalize(t_x, dim=1)*12.0

print(t_x.shape)
linearpred = linearpred.cuda()
t_y = torch.tensor(y).long().cuda()

print(t_y.shape, t_y.max(),t_y.min())

for i in range(epoch):
    
    p_y = linearpred(t_x)
    c_loss = crossentropy(p_y,t_y)
    x_cen = center_loss(t_x, t_y)
    
    loss = 0.01 * x_cen + c_loss
    
    p_y_lab = output_softmax(p_y)
    pred_one_labels = torch.max(p_y_lab, dim=1)[1]
    corrects = float((pred_one_labels.cuda() == t_y).sum().item())
    accuracy = corrects / len(y)
    
    if (i+1) % 5 == 0:
        print("Accuracy is %.4f %% " % (accuracy*100.)) # 100%
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

torch.Size([1392, 128])
torch.Size([1392]) tensor(49, device='cuda:0') tensor(0, device='cuda:0')
Accuracy is 58.5489 % 
Accuracy is 95.1868 % 
Accuracy is 98.6351 % 
Accuracy is 98.9943 % 
Accuracy is 99.6408 % 
Accuracy is 99.7845 % 
Accuracy is 99.8563 % 
Accuracy is 99.9282 % 
Accuracy is 99.9282 % 
Accuracy is 99.9282 % 
Accuracy is 99.9282 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 


In [18]:
lin_pre_x = torch.tensor(pre_x).cuda(0)
lin_pre_x = F.normalize(lin_pre_x, dim=1)*12.0

linearpred.eval()
with torch.no_grad():
    lin_pre_y = linearpred(lin_pre_x)
    lin_p_y_lab = output_softmax(lin_pre_y)
    lin_pred_labels = torch.max(lin_p_y_lab, dim=1)[1]

    lin_real_y = torch.tensor(real_y).long().cuda()

    corrects = float((lin_pred_labels.cuda() == lin_real_y).sum().item())
    lin_test_accuracy = corrects / len(real_y)
    print(lin_test_accuracy*100.) 
    
# 26.65% # cos sim include out set
# 95.2043% # vec without out set data 80epoch


95.34093435386032
